# Import libraries

In [2]:
# Importer les bibliothèques nécessaires
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

In [3]:
# Fonction pour extraire l'année manuellement avec gestion des erreurs
def extract_year_manual(date_str):
    try:
        # Vérifier si la valeur est une chaîne et non vide
        if isinstance(date_str, str) and date_str.strip():
            components = date_str.split()
            if len(components) >= 3:  # S'assurer qu'il y a au moins 3 éléments
                year_str = components[2]
                year = int(year_str)
                return year
        return None  # Retourner None si la date n'est pas valide
    except Exception as e:
        return None  # Retourner None en cas d'erreur

# Fonction pour extraire le mois manuellement avec gestion des erreurs
def extract_month_manual(date_str):
    try:
        # Vérifier si la valeur est une chaîne et non vide
        if isinstance(date_str, str) and date_str.strip():
            components = date_str.split()
            if len(components) >= 2:  # S'assurer qu'il y a au moins 2 éléments
                month_str = components[1]
                return month_str.lower()
        return None  # Retourner None si la date n'est pas valide
    except Exception as e:
        return None  # Retourner None en cas d'erreur

# Charger et préparer les données
def load_data():
    # Charger les données depuis un fichier CSV ou une autre source
    data = pd.read_csv('Tricount_Switzerland.csv')

    # Convertir les valeurs de 'Date & heure' en chaînes de caractères et gérer les valeurs manquantes
    data['Date & heure'] = data['Date & heure'].astype(str)

    # Appliquer les fonctions pour extraire l'année et le mois en gérant les valeurs manquantes
    data['Année'] = data['Date & heure'].apply(extract_year_manual)
    data['Mois'] = data['Date & heure'].apply(extract_month_manual)

    # Dictionnaire pour mapper les noms de mois en numéros de mois
    mois_map = {
        'janvier': 1, 'février': 2, 'mars': 3,
        'avril': 4, 'mai': 5, 'juin': 6,
        'juillet': 7, 'août': 8, 'septembre': 9,
        'octobre': 10, 'novembre': 11, 'décembre': 12
    }

    # Convertir la colonne 'Année' en entier et gérer les valeurs manquantes
    data['Année'] = data['Année'].fillna(0).astype(int)

    # Filtrer les lignes avec des valeurs nulles dans 'Année' et 'Mois'
    data = data.dropna(subset=['Année', 'Mois'])

    # Mapper les mois en numéros et s'assurer que les valeurs sont correctes
    data['Numéro Mois'] = data['Mois'].map(mois_map)

    # Ajouter une colonne 'Date' en combinant Année et Numéro Mois, en gérant les erreurs de conversion
    data['Date'] = pd.to_datetime(
        data['Année'].astype(str) + '-' + 
        data['Numéro Mois'].astype(str) + '-01', 
        errors='coerce'
    )

    # Supprimer les lignes où 'Date' reste NaT
    data = data.dropna(subset=['Date'])

    # Convertir les montants en valeurs absolues
    montant_cols = ['Montant', 'Montant dans la devise du tricount (CHF)', 'Payé par Caps', 'Payé par Emilian', 'Impacté à Caps', 'Impacté à Emilian']
    for col in montant_cols:
        if col in data.columns:
            data[col] = data[col].abs()

    # Sélectionner les colonnes nécessaires pour le dashboard
    columns = ["Date", "Année", "Mois", "Type de transaction", "Catégorie", "Impacté à Caps", "Impacté à Emilian"]
    df = data[columns]

    # Filtrer les lignes avec des dates valides seulement
    df = df.dropna(subset=['Date'])

    # Supprimer les lignes nulles globalement, en particulier la dernière ligne si elle est totalement vide
    df = df.dropna(how='all')

    return df

In [4]:
df = load_data()
df

,Date,Année,Mois,Type de transaction,Catégorie,Impacté à Caps,Impacté à Emilian
0,2024-05-01,2024,mai,Rentrée d'argent,Salaire 💵,0.00,180.00
1,2024-03-01,2024,mars,Rentrée d'argent,Salaire 💵,0.00,180.00
2,2024-05-01,2024,mai,Rentrée d'argent,Salaire 💵,2023.00,0.00
3,2024-04-01,2024,avril,Rentrée d'argent,Salaire 💵,2023.00,0.00
4,2024-03-01,2024,mars,Rentrée d'argent,Salaire 💵,2023.00,0.00
...,...,...,...,...,...,...,...
142,2024-03-01,2024,mars,Dépense,Restaurants & Bars,14.90,14.90
143,2024-03-01,2024,mars,Dépense,Courses,12.30,12.30
144,2024-02-01,2024,février,Dépense,Courses,20.85,20.85
145,2024-03-01,2024,mars,Dépense,Dépense unique ⚡,52.22,52.23


In [26]:
# Configuration de la page Streamlit
st.set_page_config(page_title="Dashboard de Suivi des Dépenses", layout="wide")

# Sidebar pour les filtres
st.sidebar.header("Filtres")
mois = st.sidebar.multiselect("Mois", options=df['Mois'].unique(), default=df['Mois'].unique())
annee = st.sidebar.multiselect("Année", options=df['Année'].unique(), default=df['Année'].unique())

# Filtrer les données en fonction des filtres sélectionnés
df_filtered = df[(df['Mois'].isin(mois)) & (df['Année'].isin(annee))]

# Affichage des indicateurs clés
st.title("Dashboard de Suivi des Dépenses")
col1, col2, col3 = st.columns(3)
col1.metric("Total pour Caps", f"{df_filtered['Impacté à Caps'].sum():.2f} CHF")
col2.metric("Total pour Emilian", f"{df_filtered['Impacté à Emilian'].sum():.2f} CHF")
col3.metric("Total Dépenses", f"{(df_filtered['Impacté à Caps'].sum() + df_filtered['Impacté à Emilian'].sum()):.2f} CHF")

# Répartition des dépenses par catégorie
st.header("Répartition des Dépenses par Catégorie")
depenses = df_filtered[df_filtered['Type de transaction'] == 'Dépense']
depenses_par_categorie = depenses.groupby('Catégorie')[['Impacté à Caps', 'Impacté à Emilian']].sum().sum(axis=1)
st.bar_chart(depenses_par_categorie)

# Graphique des revenus et dépenses dans le temps
st.header("Revenus et Dépenses dans le Temps")
revenus = df_filtered[df_filtered['Type de transaction'] == "Rentrée d'argent"].groupby('Date')[['Impacté à Caps', 'Impacté à Emilian']].sum()
depenses = df_filtered[df_filtered['Type de transaction'] == 'Dépense'].groupby('Date')[['Impacté à Caps', 'Impacté à Emilian']].sum()
st.line_chart(revenus - depenses)

# Épargne mensuelle
st.header("Épargne Mensuelle")
epargne_mensuelle = (revenus - depenses).sum(axis=1)
st.line_chart(epargne_mensuelle)

2024-09-06 22:11:45.238 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 22:11:45.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 22:11:45.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 22:11:45.242 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 22:11:45.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 22:11:45.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 22:11:45.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-06 22:11:45.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()